In [2]:

import cv2 as cv
import numpy as np 
import tensorflow as tf

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                rescale = 1/255.0,  # Rescale... 0 to 1 .. 0 is white colors and 1 is black colors
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.20,
                                                                horizontal_flip=True,
                                                                vertical_flip=True
                                                                )

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                rescale = 1/255.0,
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.20,
                                                                horizontal_flip=True,
                                                                vertical_flip=True
                                                                )

train_generator = train_datagen.flow_from_directory(
                                                    directory='data/train',
                                                    target_size=(224, 224),
                                                    batch_size=12,
                                                    class_mode='binary',
                                                    shuffle=True,
                                                    seed=42
                                                    )

valid_generator = valid_datagen.flow_from_directory(
                                                    directory='data/validation',
                                                    target_size=(224, 224),
                                                    batch_size=12,
                                                    class_mode='binary',
                                                    shuffle=True,
                                                    seed=42
                                                    )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [4]:
# sequential API 

# To improve accuract  i added another two Convolutional layers and also a Max Pool Layer
model = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(64, (3,3), input_shape=(224,224,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(512, activation = 'relu'),
                            tf.keras.layers.Dense(256, activation = 'relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')    
                        ])
model.summary()
# functional API

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       147584    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [5]:
model.compile(
            loss = 'binary_crossentropy',
            optimizer = 'adam',
            metrics=['accuracy']
            )

model.fit(
        train_generator,
        validation_data=valid_generator,
        epochs=20
        )

Epoch 1/20
86/86 [==============================] - 39s 451ms/step - loss: 0.6959 - accuracy: 0.5015 - val_loss: 0.6963 - val_accuracy: 0.5000
Epoch 2/20
86/86 [==============================] - 45s 518ms/step - loss: 0.6300 - accuracy: 0.6563 - val_loss: 0.7832 - val_accuracy: 0.6133
Epoch 3/20
86/86 [==============================] - 37s 423ms/step - loss: 0.4753 - accuracy: 0.8062 - val_loss: 1.9323 - val_accuracy: 0.6641
Epoch 4/20
86/86 [==============================] - 36s 416ms/step - loss: 0.3471 - accuracy: 0.8715 - val_loss: 0.8687 - val_accuracy: 0.8047
Epoch 5/20
86/86 [==============================] - 38s 446ms/step - loss: 0.3096 - accuracy: 0.8724 - val_loss: 1.5907 - val_accuracy: 0.7266
Epoch 6/20
86/86 [==============================] - 35s 409ms/step - loss: 0.2561 - accuracy: 0.8997 - val_loss: 1.1429 - val_accuracy: 0.8047
Epoch 7/20
86/86 [==============================] - 35s 409ms/step - loss: 0.2084 - accuracy: 0.9231 - val_loss: 1.3099 - val_accuracy: 0.8047

In [6]:
train_generator.class_indices

{'horses': 0, 'humans': 1}

In [7]:
def inference(image_path):
    img = cv.imread(image_path)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (224,224))
    img = img / 255.0
    img = np.array([img])
    P = model.predict(img).squeeze()
    return 'HUMAN' if P > 0.5 else 'HORSE'
    
inference('data/train/horses/horse01-9.png')

1/1 [==============================] - 0s 83ms/step


'HORSE'